# Ragas Evaluation with Llama Stack - Demo [remote execution]

This notebook demonstrates how to use the Ragas out-of-tree provider.


## Run your Llama Stack distribution

Nuke any old distro config files you might have lying around (I find these get in the way whenever I change my `.env` variables): 
```bash
ls ~/.llama/distributions/
rm -r ~/.llama/distributions/trustyai_ragas_distro
```

Then, run your llama stack server with:
```bash
dotenv run uv run llama stack run distribution/run.yaml
```




## Setup and Imports


In [1]:
# Install dev packages if not already installed
# !uv pip install -e ".[dev]"

import os
from datetime import datetime

from llama_stack_client import LlamaStackClient
from rich.pretty import pprint

## Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [2]:
# You will need ngrok to enable remote access to your Llama Stack server
client = LlamaStackClient(base_url=os.getenv("KUBEFLOW_LLAMA_STACK_URL"))
assert client.models.list()

INFO:httpx:HTTP Request: GET https://3e112b480059.ngrok-free.app/v1/models "HTTP/1.1 200 OK"


## Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [3]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## Dataset Registration

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [4]:
# Register the dataset
dataset_id = "ragas_demo_dataset_remote"

dataset_response = client.datasets.register(
    dataset_id=dataset_id,
    purpose="eval/question-answer",  # RAG evaluation purpose
    source={"type": "rows", "rows": evaluation_data},
    metadata={
        "provider_id": "localfs",  # seems there's a bug in datasets
        "description": "Sample RAG evaluation dataset for Ragas demo",
        "size": len(evaluation_data),
        "format": "ragas",
        "created_at": datetime.now().isoformat(),
    },
)
pprint(dataset_response)

INFO:httpx:HTTP Request: POST https://3e112b480059.ngrok-free.app/v1/datasets "HTTP/1.1 200 OK"


DatasetRegisterResponse(
│   identifier='ragas_demo_dataset_remote',
│   metadata={
│   │   'provider_id': 'localfs',
│   │   'description': 'Sample RAG evaluation dataset for Ragas demo',
│   │   'size': 3.0,
│   │   'format': 'ragas',
│   │   'created_at': '2025-09-25T19:26:49.309288'
│   },
│   provider_id='localfs',
│   purpose='eval/question-answer',
│   source=SourceRowsDataSource(
│   │   rows=[
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   type='rows'
│   ),
│   type='dataset',
│   provider_resource_id='ragas_demo_dataset_remote',
│   owner=None
)

## Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [5]:
benchmark_id = "ragas_demo_benchmark_remote"

benchmark_response = client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=dataset_id,
    scoring_functions=[
        "answer_relevancy",  # How relevant is the answer to the question?
        # "context_precision",     # How precise are the retrieved contexts?
        # "faithfulness",          # How faithful is the answer to the contexts?
        # "context_recall",        # How much of the ground truth is covered by contexts?
        # "answer_correctness"  # How correct is the answer compared to ground truth?
    ],
    provider_id="trustyai_ragas",
    # metadata={
    #     "provider": "ragas",
    #     "version": "1.0",
    #     "metrics_count": len(ragas_metrics),
    #     "created_at": datetime.now().isoformat()
    # }
)

pprint(benchmark_response)

INFO:httpx:HTTP Request: POST https://3e112b480059.ngrok-free.app/v1/eval/benchmarks "HTTP/1.1 200 OK"


None

In [6]:
benchmarks = client.benchmarks.list()
pprint(benchmarks[-1:])

INFO:httpx:HTTP Request: GET https://3e112b480059.ngrok-free.app/v1/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset_remote',
│   │   identifier='ragas_demo_benchmark_remote',
│   │   metadata={},
│   │   provider_id='trustyai_ragas',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark_remote'
│   )
]

## Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [7]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

job = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(job)

INFO:httpx:HTTP Request: POST https://3e112b480059.ngrok-free.app/v1/eval/benchmarks/ragas_demo_benchmark_remote/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='afc31c92-5887-4b84-933c-009e8ae0594e',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark_remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark_remote',
│   │   │   'provider_id': 'trustyai_ragas',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset_remote',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline/test-two',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-dspa-ragas-eval-v3.apps.rosa.diego-ragas.zch5.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-v3',
│   │   │   'llama_stack_url': 'https://3e112b480059.ngrok-free.app',
│   │   │   'base_image': 'quay.io/diegosquayorg/my-ragas-provider-image:latest'
│   │   }
│   },
│   kubeflow_run_id='738a0fcf-8780-4a9d-b661-0874de18b96c',
│   result=None
)

## Results Display


In [8]:
job = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(job)

INFO:httpx:HTTP Request: GET https://3e112b480059.ngrok-free.app/v1/eval/benchmarks/ragas_demo_benchmark_remote/jobs/afc31c92-5887-4b84-933c-009e8ae0594e "HTTP/1.1 200 OK"


Job(
│   job_id='afc31c92-5887-4b84-933c-009e8ae0594e',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark_remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark_remote',
│   │   │   'provider_id': 'trustyai_ragas',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset_remote',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline/test-two',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-dspa-ragas-eval-v3.apps.rosa.diego-ragas.zch5.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-v3',
│   │   │   'llama_stack_url': 'https://3e112b480059.ngrok-free.app',
│   │   │   'base_image': 'quay.io/diegosquayorg/my-ragas-provider-image:latest'
│   │   }
│   },
│   kubeflow_run_id='738a0fcf-8780-4a9d-b661-0874de18b96c',
│   result=None
)

In [11]:
# wait a bit for the job to complete
pprint(client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id))

INFO:httpx:HTTP Request: GET https://3e112b480059.ngrok-free.app/v1/eval/benchmarks/ragas_demo_benchmark_remote/jobs/afc31c92-5887-4b84-933c-009e8ae0594e "HTTP/1.1 200 OK"


Job(
│   job_id='afc31c92-5887-4b84-933c-009e8ae0594e',
│   status='completed',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark_remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark_remote',
│   │   │   'provider_id': 'trustyai_ragas',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset_remote',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline/test-two',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-dspa-ragas-eval-v3.apps.rosa.diego-ragas.zch5.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-v3',
│   │   │   'llama_stack_url': 'https://3e112b480059.ngrok-free.app',
│   │   │   'base_image': 'quay.io/diegosquayorg/my-ragas-provider-image:latest'
│   │   }
│   },
│   kubeflow_run_id='738a0fcf-8780-4a9d-b661-0874de18b96c',
│   result={
│   │   'generations': [
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   'scores': {
│   │   │   'answer_relevancy': {
│   │   │   │   'score_rows': [
│   │   │   │   │   {'score': 0.9567410688},
│   │   │   │   │   {'score': 0.9262221944000001},
│   │   │   │   │   {'score': 0.8774825363000001}
│   │   │   │   ],
│   │   │   │   'aggregated_results': {
│   │   │   │   │   'average': 0.9201485998333334,
│   │   │   │   │   'count': 3,
│   │   │   │   │   'min': 0.8774825363000001,
│   │   │   │   │   'max': 0.9567410688
│   │   │   │   }
│   │   │   }
│   │   }
│   }
)

In [12]:
results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(results)

INFO:httpx:HTTP Request: GET https://3e112b480059.ngrok-free.app/v1/eval/benchmarks/ragas_demo_benchmark_remote/jobs/afc31c92-5887-4b84-933c-009e8ae0594e/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What is the capital of France?',
│   │   │   'response': 'The capital of France is Paris.',
│   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   'reference': 'Paris'
│   │   },
│   │   {
│   │   │   'user_input': 'Who invented the telephone?',
│   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   ],
│   │   │   'reference': 'Alexander Graham Bell'
│   │   },
│   │   {
│   │   │   'user_input': 'What is photosynthesis?',
│   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   ],
│   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   }
│   ],
│   scores={
│   │   'answer_relevancy': ScoringResult(
│   │   │   aggregated_results={
│   │   │   │   'average': 0.9201485998333334,
│   │   │   │   'count': 3.0,
│   │   │   │   'min': 0.8774825363000001,
│   │   │   │   'max': 0.9567410688
│   │   │   },
│   │   │   score_rows=[{'score': 0.9567410688}, {'score': 0.9262221944000001}, {'score': 0.8774825363000001}]
│   │   )
│   }
)